# Introduction

This notebook contains all the code needed to automatically calculate the Facial-Width-Height ratio based on an image using the `face_recognition` package.  
**See: https://github.com/ageitgey/face_recognition**

In [2]:
NAME = 'FWHR_calculator'
PYTHON_VERSION = '3.5'
AUTHOR = 'Ties de Kok'
MODIFIED BY = 'Carl Lindersson'

# Modification ideas - By Carl

#### 1. Rotate images to better fit the horisontal lines. - Enables more accurate reading of fWHR 
####    - minimize the hight differens between left and right side. 
#### 2. Store and orginize data automatically in existing numbers/excell file. - Enables more rapid testing and less human error variation.
####    - Investigate the pipeline, note steps of data aquisition 
####    - Note steps needing to be automated
####    - Write psudo code for the general pipeline of the project 
####    - Write psudo code for each step in particular 
####    - Write the code following your psudo code. 
#### 3. Find sutable error check calibrations - Enable more accurate reading of large data and minimise human error variation 
####    - Test problematic, slightly problematic and good images to find a good cut-off point. 
3. Script to get fWHR from list of images - DONE 
4. Print image with box outlined - enabele rapid error check - DONE 
5. Tweak face points used to better reflect fWHR - Enable more accurate measures - In progress 


In [184]:
##### Scrape Stats, Names, Image URLs and Images from web

#Stats saved in: 
#Images saved in: 

import bs4
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import requests
import pandas as pd 
import json 
from selenium import webdriver
import urllib.request
from openpyxl import load_workbook

def ScrapePlayerInfo(TeamURL):
#--------------- Selenium Experiment --------------
    #cookies = dict(BCPermissionLevel='PERSONAL')
        #driver = webdriver.Chrome("/Users/carllindersson/Documents/Neurovetenskap/Programs/chromedriver")
    # Set chrome as head driver in selenium
        #driver = webdriver.Firefox()
    # set teamURLs as the webbpage that the driver goes to 
        #driver.get(TeamURL)
    # Variable for the webpage (where the driver has navigated)
       # html = driver.page_source
    # variable for Beautifulsoup parsing the website the driver has navigated to.
       # page_soup = soup(html)
#--------------------------------------------------   
   # Bypass cookie agreement  
    from http import cookiejar  
    class BlockAll(cookiejar.CookiePolicy):
        return_ok = set_ok = domain_return_ok = path_return_ok = lambda self, *args, **kwargs: False
        netscape = True
        rfc2965 = hide_cookie2 = False
    
    s = requests.Session()
    s.cookies.set_policy(BlockAll())
#--------------------------------------------------
    
    # set Team webpage as source for data collection / scrape 
    source = s.get(TeamURL).text
    # set up BeautifulSoup to parse the source website using the html.parser
    page_soup = soup(source, 'html.parser')

    # Variables indicating points of interests on the webpage by the page's html tags 
  
    Table = page_soup.find('div', class_="ResponsiveTable ResponsiveTable--fixed-left mt5 remove_capitalize")
    Names = Table.find('tbody', class_="Table__TBODY")

    Stat_table = Table.find("div", class_="Table__Scroller")
    table_row = Stat_table.find('tr', class_='Table__TR Table__TR--sm Table__even')

    # Name and statistics lists made from elements with same tag 
    Name_tag_list = Names.find_all('a', class_='AnchorLink')
    Stat_row_tag_list = Stat_table.find_all('tr', class_='Table__TR Table__TR--sm Table__even')

    #create empty lists to store data in 
    Name_list = []
    Stat_list = []
    Name_Stat_list = []

    #looping through the names and stats, adding them to the list Names_Stat_list
    for Name, row in zip(Name_tag_list, Stat_row_tag_list):
        Name_list.append(str(Name.text).replace('  ',' ')) # list each player's names
        Team_name_tag = page_soup.find('div', class_="ClubhouseHeader__Main_Aside pl4 relative").find('h1').text
        Name_list.append(Team_name_tag)
        for stat in row:
            Stat_list.append(str(stat.text)) # lists each inner iteration, the stats for each player 
        Name_Stat_list.append(Name_list + Stat_list)  # appends the stats for each player as specific elements in a separate list.  
        Stat_list = [] # clear Stat_list so that the inner iteration does not contain the previous players stats. 
        Name_list = [] # clear Name_list so that the outer iteration does not contain the previous players Name or Team.

    # Create dataframe from Name_stat_list
    df_Name_Stat = pd.DataFrame(Name_Stat_list)
    df_Name_Stat.rename(columns = {0: 'Name',1:'Team'}, inplace=True)

    #Set column names using list of stat names 
        #1. Create list of stat headings 
    Table_headings_list = page_soup.find_all('th', class_="tar stats-cell Table__TH")[0:13]
    Heading_name_list = ['Name', 'Team'] # 'Name' was not in the Table heading list and needed to be manually added.
    for heading in Table_headings_list:
        Heading_name_list.append(heading.text)

        #2. Assign list as the column names of the dataframe
    df_Name_Stat.columns = Heading_name_list


    #-----------------------------------------------------------------------------------------
    # Loop through list of player bios and extract their weight, height, image url, image and name.

    # list with all player profile links 
    Name_link_list = []
    for info in Name_tag_list: 
        Name_link_list.append(info.get('href'))
    
    
    All_Content_list = []
    Content_list = []
    for link in Name_link_list:
        # Find html tags for picture url, hight, and weight 
        New_source = requests.get(link).text
        page_soup = soup(New_source, 'html.parser')
        Content_list = []

        if page_soup.find('div', class_="main-headshot-65 floatleft"):
            Main_url_tag = page_soup.find('div', class_="main-headshot-65 floatleft")
            Url_tag = Main_url_tag.find('img').get('src')
            Content_list.append(Url_tag)
            Name2_tag = Main_url_tag.find('img').get('alt') # name to later controll of it matches the other dataframe names. 
            Content_list.append(Name2_tag)
            Height_Weight_tag = page_soup.find('ul', class_="general-info").find('li').next_sibling.text.split(',')
            Height = Height_Weight_tag[0].replace('"', '').replace("' ", '').replace("' ", '')
            feet = int(Height[0])
            inches = int(Height[1:2])
            feet_in_inches = feet * 12
            Height_centimeters = (feet_in_inches + inches) * 2.54 # Convert feet and inches to centimeters 
            Content_list.append(Height_centimeters)
            Weight = int(Height_Weight_tag[1].replace('lbs','')) * 0.45359237   # Convert lbs to kg 
            Content_list.append(Weight)
            Position_tag = page_soup.find('ul', class_="general-info").find('li', class_='first').text[-2:]#Player's position 
            Content_list.append(Position_tag)
        elif page_soup.find('div', class_="main-headshot"): # Other layout of page with other tag
            Main_url_tag = page_soup.find('div', class_="main-headshot")
            Url_tag = Main_url_tag.find('img').get('src')
            Content_list.append(Url_tag)
            Name2_tag = Main_url_tag.find('img').get('alt') # name to later controll of it matches the other dataframe names. 
            Content_list.append(Name2_tag)
            Height_Weight_tag = page_soup.find('ul', class_="general-info").find('li').next_sibling.text.split(',')
            Height = Height_Weight_tag[0].replace('"', '').replace("' ", '').replace("' ", '')
            feet = int(Height[0])
            inches = int(Height[1:2])
            feet_in_inches = feet * 12
            Height_centimeters = (feet_in_inches + inches) * 2.54
            Content_list.append(Height_centimeters)
            Weight = int(Height_Weight_tag[1].replace('lbs','')) * 0.45359237   # Convert lbs to kg 
            Content_list.append(Weight)
            Position_tag = page_soup.find('ul', class_="general-info").find('li', class_='first').text[-2:]#Player's position 
            Content_list.append(Position_tag)
        else:
            pass
        All_Content_list.append(Content_list)
 
    df_profile = pd.DataFrame(All_Content_list) # Create dataframe from list of player bio content.
    df_profile.rename(columns = {0: 'Image URL',1: 'Name',2: 'Height(cm)',3: 'Weight(kg)',4: 'Position'}, inplace=True) # Set heading names
#------------------------------------------------------
        #Downloads images from urls in df.profile  
    
    df_Image_Download = df_profile[['Name','Image URL']] # creates dataframe only containing name and url 
    
    for i,j in df_Image_Download.itertuples(index=False): # i becomes first column (Name) and j the second (Image URL). Index is ignored.
        if not isinstance(i, str)==True: 
            filename = 'unknown'
        else:
            filename = i # save jpg as the current player name column 
        file_path = '/Users/carllindersson/Documents/Neurovetenskap/fWHR_Summer/Behaviour_and_statistics/Scraped_Data/Photos/test_table_photos/'
        full_path = '{}{}'.format(file_path, filename)
        if not isinstance(j, str)==True: # if j (the URL column element) is not a string, and hence, not a url, move on.  
            continue
        else: 
            urllib.request.urlretrieve(j, full_path) # download file from url string and save it in/as full_path
          # print(filename,"saved.")
#------------------------------------------------------  

#-----------------------------------------------------------------------------------------
    # Combine both Name_stat and Player bio DataFrames while matching the rows using the name columns as indicators
    df_complete = df_Name_Stat.join(df_profile.set_index('Name'), on='Name') 
    #print(df_complete)
    
    # Append df_complete to a csv that later will contain all teams' statistics
    df_complete.to_csv('/Users/carllindersson/Documents/Neurovetenskap/fWHR_Summer/Behaviour_and_statistics/Scraped_Data/Statistics/All_but_fWHR.csv', mode = 'a', header=False) 
#-----------------------------------------------------------------------------------------

#create csv file to later append fwhr-statistics to. 
csv_header_list = ['Team Index','Name','Team','GP','G','A','PTS','+/-','PIM','TOI/G','PPG','PPA','SHG','SHA','S','SPCT','Image URL','Height(cm)','Weight(kg)','Position']
import csv
with open('/Users/carllindersson/Documents/Neurovetenskap/fWHR_Summer/Behaviour_and_statistics/Scraped_Data/Statistics/All_but_fWHR.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(csv_header_list)
    
#--------------------------------------------------
#Gets fwhr on the images and then add the correct ratio to the correct name in the statistics files. 
import os
import pandas as pd

#create list to later append data to 
fwhr_list = []
#list of faces from directory
faces_list = [f for f in os.listdir("/Users/carllindersson/Documents/Neurovetenskap/fWHR_Summer/Behaviour_and_statistics/Scraped_Data/Photos/test_table_photos") if not f.startswith('.')]
#loop through list, get fwhr and append with player name to df_fwhr
for face in faces:
    os.chdir("/Users/carllindersson/Documents/Neurovetenskap/fWHR_Summer/Behaviour_and_statistics/Scraped_Data/Photos/test_table_photos")
    fWHR_Eyebrow = get_fwhr(face, show=False, top='eyebrow')
    fWHR_Eyelid = get_fwhr(face, show=False, top='eyelid')
    row_list = [str(face),str(fWHR_Eyebrow),str(fWHR_Eyelid)]
    fwhr_list.append(row_list)
    #df_row = pd.DataFrame(row_list)
    #df_fwhr.append(df_row)

df_fWHR = pd.DataFrame(fwhr_list, columns=['Name','fWHR Eyebrow', 'fWHR Eyelid'])


#df_fwhr = pd.read_csv('playerfwhr.csv', names=["Name","fWHR Eyebrow","fWHR Eyelid"], index_col=False)

    
#--------------------------------------------------
    
#list of Team Pages to scrape 
Team_page_list = ['https://www.espn.com/nhl/team/stats/_/name/tor/season/2008/seasontype/2','https://www.espn.com/nhl/team/stats/_/name/cgy/season/2008/seasontype/2','https://www.espn.com/nhl/team/stats/_/name/edm/season/2008/seasontype/2','https://www.espn.com/nhl/team/stats/_/name/mtl/season/2008/seasontype/2','https://www.espn.com/nhl/team/stats/_/name/ott/season/2008/seasontype/2','https://www.espn.com/nhl/team/stats/_/name/van/season/2008/seasontype/2']

# loop that calls the scrape funciton on the team page list. 
for TeamPage in Team_page_list:
    ScrapePlayerInfo(TeamPage)
    
# Add df_fwhr to the dataframe and turn the final fully appended csv-file to an excel file
df_all_but_fwhr = pd.read_csv('/Users/carllindersson/Documents/Neurovetenskap/fWHR_Summer/Behaviour_and_statistics/Scraped_Data/Statistics/All_but_fWHR.csv')
df_final = df_all_but_fwhr.join(df_fWHR.set_index('Name'), on='Name')
writer = pd.ExcelWriter('/Users/carllindersson/Documents/Neurovetenskap/fWHR_Summer/Behaviour_and_statistics/Scraped_Data/Statistics/Complete_Table.xlsx')
df_final.to_excel(writer, index = False)
writer.save()



In [188]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

#writer2 = pd.csvWriter('/Users/carllindersson/Documents/Neurovetenskap/fWHR_Summer/Behaviour_and_statistics/Scraped_Data/Statistics/Complete_Table.xlsx')
df_final.to_csv('/Users/carllindersson/Documents/Neurovetenskap/fWHR_Summer/Behaviour_and_statistics/Scraped_Data/Statistics/Complete_Table.csv', index = False)


In [169]:
##### Structure and organise data
## Step 1 - Get names and images through wget
# 1. Select file and header
import pandas as pd 
import numpy as np
import urllib.request
df = pd.read_excel('/Users/carllindersson/Documents/Neurovetenskap/fWHR_Summer/Behaviour_and_statistics/Scraped_Data/Names_and_Image_URLs/test_table.xlsx', header=0)
  # show all rows in dataframe
pd.set_option('display.max_rows', None)
# 2. copy name2 and name3 to and paste to Name1 column / replace empty spaces in Name1 with names from Name2 and 3 (test on name 2 to 3)
    #### OBS! The This code ONLY adjust for Name1-3 and Image_URL1-3. If more colums are used, change code. 
if 'Name2' in df.columns:
    df['Name1'] = df['Name1'].fillna(df.Name2)
if 'Name3' in df.columns:
    df['Name1'] = df['Name1'].fillna(df.Name3)
# 3. copy URL2 and URL3 and paste to URL1 column 
if "Image_URL2" in df.columns:
    df['Image_URL1'] = df['Image_URL1'].fillna(df.Image_URL2)
if "Image_URL3" in df.columns:
    df['Image_URL1'] = df['Image_URL1'].fillna(df.Image_URL3)  
# 4. Create data frame with only Name1 and Image_URL1 
df2 = df[["Name1", "Image_URL1"]]

#-------------------------------------
#   Only needed for using wget and possibly later error checks. 

# 5. Save Name1 and URL1 as "file_name_pruned"
df2.to_excel('/Users/carllindersson/Documents/Neurovetenskap/fWHR_Summer/Behaviour_and_statistics/Scraped_Data/Names_and_Image_URLs/test_table_pruned.xlsx')
# 6. Remove spaces and special characters (') from colums, saving new data frame as "df3" 
df3 = pd.DataFrame(df2)
df3['Name1'] = df2['Name1'].str.replace(' ', '')
df3['Name1'] = df2['Name1'].str.replace("'", "")
df3['Image_URL1'] = df2['Image_URL1'].str.replace(' ', '')
df3['Image_URL1'] = df2['Image_URL1'].str.replace("'", "")
# 7. Move df3 to .txt file, remove headers and index, and save in the folder for names and urls
df3.to_string('/Users/carllindersson/Documents/Neurovetenskap/fWHR_Summer/Behaviour_and_statistics/Scraped_Data/Names_and_Image_URLs/test_table_wget', header=False, index=False)
    # also a csv file to see if it is better. 
df3.to_csv('/Users/carllindersson/Documents/Neurovetenskap/fWHR_Summer/Behaviour_and_statistics/Scraped_Data/Names_and_Image_URLs/test_table_download', header=False, index=False)
#-------------------------------------
        #Downloads images from urls in df2 
    
# 8. run wget on text/cvs-file / or use python code to get images from df2 
   
    
for i,j in df2.itertuples(index=False): # i becomes first column (Name1) and j the second (Imaeg_URL1). Index is ignored.
    if not isinstance(i, str)==True: 
        filename = unknown
    else:
        filename = i # save jpg as the current player name column 
    file_path = '/Users/carllindersson/Documents/Neurovetenskap/fWHR_Summer/Behaviour_and_statistics/Scraped_Data/Photos/test_table_photos/'
    full_path = '{}{}'.format(file_path, filename)
    if not isinstance(j, str)==True: # if j (the URL column element) is not a string, and hence, not a url, move on.  
        continue
    else: 
        urllib.request.urlretrieve(j, full_path) # download file from url and save it in/as full_path
#        print(filename,"saved.")
#--------------------------------------   


# 9. save images into specific image folder called "Sample_Name_Images" OBS! Make sure it can be found by the fWHR_calculator-code later. 

## 2. Organise stats, names and images
# 1. Combine stat sheet with image/Name data 




KeyboardInterrupt: 

In [ ]:
##### use python code to get images

#------------------------------ 

def URL_to_jpg(i, url, file_path):
    
    filename = '{}.jpg'.format(urls['Name1'])
    full_path = '{}{}'.format(file_path, filename)
    urllib.request.urlretrieve(url, full_path)
    
    print(' {} saved.'.format(filename))
    
    return None 
#----------------------------
    # constants
FILENAME = 'test_table_pruned.xlsx'
FILEPATH = '/Users/carllindersson/Documents/Neurovetenskap/fWHR_Summer/Behaviour_and_statistics/Scraped_Data/Photos/test_table_photos'
#----------------------------
#read list of urls as panda data frame 
urls = pd.read_excel(FILENAME)

# Save images in chosen directory by iterating through list
if i in urls:
    for i, url in urls['Image_URL1']:
        URL_to_jpg(i, url[1], FILEPATH)
    



## Looking for examples? Click here: [Examples](#examples)

# Preamble

## Imports

In [ ]:
%pylab inline

In [137]:
import math

from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt #####* added by Carl to use plt.show() in the show_box function 
from PIL import Image, ImageDraw
import face_recognition

import urllib.request 

**All the heavy-lifting is done by the `face_recognition` package, you need to have it installed!**  
**See: https://github.com/ageitgey/face_recognition**

# Main Code

### Function to load an image

This function loads an image from the drive or downloads it from a link if `url` is set to `True`.

In [138]:
def load_image(path, url=False):
    if not url:
        return face_recognition.load_image_file(path)
    else:
        if path[-3:] == 'jpg' or  path[-3:] == 'peg': #######Change to -4 to jpeg to make it more intuitive
            urllib.request.urlretrieve(path, 'tmp.jpg')
            return face_recognition.load_image_file('tmp.jpg')
        elif path[-3:] == 'png':
            urllib.request.urlretrieve(path, 'tmp.png')
            return face_recognition.load_image_file('tmp.png')
        else:
            print("Unknown image type")

### Function to get the contour face points

This function calculates the coordinates for the corners of the "FWHR" box.  
*Note 1:* it is possible to calculate the top line based on either the bottom of the eyebrows (`top = "eyebrow"`) or the eyelids (`top = "eyelid"`).  
*Note 2:* to counter-act small amounts of rotation it will by default take the average between the height of the two top points.

In [139]:
def get_face_points(points, method='average', top='eyebrow'):
    width_left, width_right = points[0], points[16] #####* Chaged by Carl to optimise performance
    
    if top == 'eyebrow':
        top_left = points[21] #####* Chaged by Carl to optimise performance
        top_right = points[22] #####* Chaged by Carl to optimise performance
        
    elif top == 'eyelid':
        top_left = points[37]
        top_right = points[43] 
        
    else:
        raise ValueError('Invalid top point, use either "eyebrow" or "eyelid"')
        
    bottom_left, bottom_right = points[50], points[52]
    
    if method == 'left':
        coords = (width_left[0], width_right[0], top_left[1], bottom_left[1])
        
    elif method == 'right':
        coords = (width_left[0], width_right[0], top_right[1], bottom_right[1])
        
    else:
        top_average = int((top_left[1] + top_right[1]) / 2)
        bottom_average = int((bottom_left[1] + bottom_right[1]) / 2)
        coords = (width_left[0], width_right[0], top_average, bottom_average)
        
    ## Move the line just a little above the top of the eye to the eyelid    
#####*    if top == 'eyelid':
#####*        coords = (coords[0], coords[1], coords[2] - 4, coords[3])      
    #####* Above Changed by Carl to optimise performance. If noted code above is performed the top==eyelid setting is too high.

    return {'top_left' : (coords[0], coords[2]),
            'bottom_left' : (coords[0], coords[3]),
            'top_right' : (coords[1], coords[2]),
            'bottom_right' : (coords[1], coords[3])
           }

### Function to check suitability of picture

This function checks whether a picture contains a person that is looking straight at the camera.

In [140]:
def good_picture_check(p, debug=False):
    ## To scale for picture size
    width_im = (p[16][0] - p[0][0]) / 100 
    
    ## Difference in height between eyes
    eye_y_l = (p[37][1] + p[41][1]) / 2.0
    eye_y_r = (p[44][1] + p[46][1]) / 2.0
    eye_dif = (eye_y_r - eye_y_l) / width_im
    
    ## Difference top / bottom point nose 
    nose_dif = (p[30][0] - p[27][0]) / width_im
    
    ## Space between face-edge to eye, left vs. right
    left_space = p[36][0] - p[0][0]
    right_space = p[16][0] - p[45][0]
    space_ratio = left_space / right_space
    
    if debug:
        print(eye_dif, nose_dif, space_ratio)
    
    ## These rules are not perfect, determined by trying a bunch of "bad" pictures
    if eye_dif > 5 or nose_dif > 3.5 or space_ratio > 3:
        return False
    else:
        return True

### Function to calculate the FWHR

This function calculates the FWHR based on the corners: `Width / Height`

In [141]:
def FWHR_calc(corners):
    width = corners['top_right'][0] - corners['top_left'][0]
    height = corners['bottom_left'][1] - corners['top_left'][1]
    return float(width) / float(height)


### Function to show the FWHR box on the image

This function shows the FWHR box on the original image.

In [142]:
def show_box(image, corners):
    pil_image = Image.fromarray(image)
    w, h = pil_image.size
    
    ## Automatically determine width of the line depending on size of picture
    line_width = math.ceil(h / 100)
    
    d = ImageDraw.Draw(pil_image) 
    d.line([corners['bottom_left'], corners['top_left']], width = line_width)
    d.line([corners['bottom_left'], corners['bottom_right']], width = line_width)
    d.line([corners['top_left'], corners['top_right']], width = line_width)
    d.line([corners['top_right'], corners['bottom_right']], width = line_width)
    
    #####* Added by Carl - Show line for brow angle
   # d.line([points[19], points[21]], width = line_width) #####* Left brow angle 
   # d.line([points[44], points[42]], width = line_width) #####* Right brow angle 
    
    imshow(pil_image)
    plt.show() #####* added by Carl to enable that the image shows for each time the function is called in a loop 

### Final function that combines the previous functions

This function combines all the previous logic into one function.  

Arguments:

`image_path` $\rightarrow$ path or URL to image  
`url` $\rightarrow$ set to `True` if `image_path` is a url  
`show` $\rightarrow$ set to `False` if you only want it to return the FWHR  
`method` $\rightarrow$ determines which eye to use for the top point: `left`, `right`, or `average`  
`top` $\rightarrow$ determines whether to use the `eyebrow` as top point or `eyelid` as top point  


In [143]:
def get_fwhr(image_path, url=False, show=True, method='average', top='eyebrow'):
    image = load_image(image_path, url)
    landmarks = face_recognition.api._raw_face_landmarks(image)
    landmarks_as_tuples = [(p.x, p.y) for p in landmarks[0].parts()]
    
    if good_picture_check(landmarks_as_tuples): 
        corners = get_face_points(landmarks_as_tuples, method=method, top = top)
        fwh_ratio = FWHR_calc(corners)
        
        if show:
            print('The Facial-Width-Height ratio is: {}'.format(fwh_ratio))
            show_box(image, corners)
            
        else:
            return fwh_ratio
    else:
        if show:
            print("Picture is not suitable to calculate fwhr.")
            imshow(image)
        else:
            return None

# Examples
<a id='examples'></a>

### Structure Scraped Data - Automate data aquisition and organisation after scrape. 
1. Get images from scraped URLs using wget 
2. Create table of names and stats combined
3. Save fWHR measure to the name/stat-list


In [350]:
##### 1. Get images from scraped URLs 

# 1. Remove spaces from name/URL-list 
#    1. Create folder and file where Scraped Names, Image URLs, and Stat data will be saved. 
#    2. Create two test-files for the two outputs from scrape storm. These will be used to test automation code.
#    3. Write code that combines the name columns to one column 
            # Might need to covert to CVS or text file 
#    4. Code that combines the image URL columns to one column 
#    5. Code that remove spaces in the name column
#    6. Code that removes special characters such as ' , " etc... 
#    7. Code that create text file with names and image URLs, saved in appropriate name/URL folder.
# 2. Use wget to download the images from the URLs 
#    8. Code that use xargs wget to download the images
#    9. Create folder for images 
#    9. Code that moves the images into a sample-specific folder and then to the approprate image folder. 


In [351]:
##### 1. Get images from scraped URLs 

# 1. Remove spaces from name/URL-list 
#    1. Create folder and file where Scraped Names, Image URLs, and Stat data will be saved. 
#    2. Create two test-files for the two outputs from scrape storm. These will be used to test automation code.
#    3. Write code that combines the name columns to one column 
            # Might need to covert to CVS or text file 
import pandas as pd 
df = pd.read_excel ('/Users/carllindersson/Documents/Neurovetenskap/fWHR_Summer/Behaviour_and_statistics/Scraped_Data/Names_and_Image_URLs/test_table.xlsx', header=[1]) #select file and header (row 1)   
# select the name2 and name3 columns and copy 
# Paste to name1 column
# select URL2 and URL3 columns and copy 
# Paste to URL1 column
# Delete all columns except name1 and URL1 
# Save file as "file_name_pruned "


## Displays fWHR box-outline, fWHR and file name of image
*program written by carl* 

In [352]:
import os

images_names_list = os.listdir('/Users/carllindersson/Documents/Neurovetenskap/fWHR_Summer/Behaviour_and_statistics/Canada_ESPN_Images/')
full_path_list = ['/Users/carllindersson/Documents/Neurovetenskap/fWHR_Summer/Behaviour_and_statistics/Canada_ESPN_Images/' + x for x in images_names_list]
for image_paths in full_path_list:
   # print(image_paths[103:])
    get_fwhr(image_paths, url=False, show=True, method='average', top='eyebrow')
    


NameError: name 'load_image' is not defined

# Gets fWHR of images
**Saves fWHR of Images together with their file names as playerfwhr.txt and playerfwhr.cvc**

*written by Oskar and Carl*

In [158]:
import os, numpy as np
import pandas as pd
os.chdir("/Users/carllindersson/Documents/Neurovetenskap/fWHR_Summer/Behaviour_and_statistics/Scraped_Data/Statistics")
with open('playerfwhr.csv', 'w') as finallist:
    os.chdir("/Users/carllindersson/Documents/Neurovetenskap/fWHR_Summer/Behaviour_and_statistics/Scraped_Data/Photos/test_table_photos")
    faces = [f for f in os.listdir("/Users/carllindersson/Documents/Neurovetenskap/fWHR_Summer/Behaviour_and_statistics/Scraped_Data/Photos/test_table_photos") if not f.startswith('.')]
    #faces = os.listdir()
    for face in faces:
        fWHR_Eyebrow = get_fwhr(faces, show=False, top='eyebrow')
        
  #  fWHR_Eyebrow = [get_fwhr(x, show=False, top='eyebrow') for x in faces]
  #  fWHR_Eyelid = [get_fwhr(x, show=False,top='eyelid') for x in faces]
  #  results = [faces,fWHR_Eyebrow,fWHR_Eyelid]
    #results = np.column_stack((faces,fWHR_Eyebrow,fWHR_Eyelid))
    finallist.write("%s\n" % results)
    
#os.chdir("/Users/carllindersson/Documents/Neurovetenskap/fWHR_Summer/Behaviour_and_statistics/Scraped_Data/Statistics")
#with open('playerfwhr.txt', 'r') as infile, open('playerfwhr.csv', 'w') as outfile:
#    data = infile.read()
#    data = data.replace("[[", "")
#    data = data.replace(" [", "")
#    data = data.replace("]", "")
#    data = data.replace(" [0:9]", ",")
#    data = data.replace("'", "")
#    outfile.write(data)
os.chdir("/Users/carllindersson/Documents/Neurovetenskap/fWHR_Summer/Behaviour_and_statistics/Scraped_Data/Statistics")
df_fwhr = pd.read_csv('playerfwhr.csv', names=["Name","fWHR_Eyebrow","fWHR_Eyelid"], index_col=False)
df_fwhr


IndexError: list index out of range

In [183]:
import os
import pandas as pd

#create list to later append data to 
fwhr_list = []
#list of faces from directory
faces_list = [f for f in os.listdir("/Users/carllindersson/Documents/Neurovetenskap/fWHR_Summer/Behaviour_and_statistics/Scraped_Data/Photos/test_table_photos") if not f.startswith('.')]
#loop through list, get fwhr and append with player name to df_fwhr
for face in faces:
    os.chdir("/Users/carllindersson/Documents/Neurovetenskap/fWHR_Summer/Behaviour_and_statistics/Scraped_Data/Photos/test_table_photos")
    fWHR_Eyebrow = get_fwhr(face, show=False, top='eyebrow')
    fWHR_Eyelid = get_fwhr(face, show=False, top='eyelid')
    row_list = [str(face),str(fWHR_Eyebrow),str(fWHR_Eyelid)]
    fwhr_list.append(row_list)
    #df_row = pd.DataFrame(row_list)
    #df_fwhr.append(df_row)

df_fWHR = pd.DataFrame(fwhr_list, columns=['Name','fWHR Eyebrow', 'fWHR Eyelid'])
print(fwhr_list)



[['Aaron Downey', '1.9523809523809523', '2.342857142857143'], ['Aaron Miller', '1.9615384615384615', '2.217391304347826'], ['Adrian Aucoin', '1.68', '1.9534883720930232'], ['Ales Hemsky', '1.803921568627451', '2.090909090909091'], ['Alex Burrows', '1.8444444444444446', '2.1842105263157894'], ['Alex Foster', '1.9318181818181819', '2.361111111111111'], ['Alex Kovalev', '1.7291666666666667', '1.930232558139535'], ['Alex Tanguay', '1.9111111111111112', '2.15'], ['Alexander Edler', '1.816326530612245', '2.022727272727273'], ['Alexander Perezhogin', '1.8181818181818181', '2.0'], ['Alexander Steen', '1.74', '2.0232558139534884'], ['Alexei Ponikarovsky', '2.022222222222222', '2.275'], ['Anders Eriksson', '1.9', '2.375'], ['Andrei Kostitsyn', '1.6226415094339623', '1.8695652173913044'], ['Andrei Markov', '1.875', '2.3076923076923075'], ['Andrew Cogliano', '1.7608695652173914', '2.025'], ['Andy Wozniewski', '1.76', '2.0'], ['Antoine Vermette', '1.8695652173913044', '2.097560975609756'], ['Anton 

## Difference `eyelid` vs. `eyebrow`

In [ ]:
obama_url =  'https://www.biography.com/.image/ar_1:1%2Cc_fill%2Ccs_srgb%2Cg_face%2Cq_auto:good%2Cw_300/MTE4MDAzNDEwNzg5ODI4MTEw/barack-obama-12782369-1-402.jpg'
get_fwhr(obama_url, url=True, top = 'eyelid')

In [ ]:
get_fwhr(obama_url, url=True, top = 'eyebrow')

## FWHR only

In [ ]:
get_fwhr(obama_url, url=True, top = 'eyebrow', show=False)

## Detection of bad picture

In [ ]:
bad_img_url_1 = 'http://s-media-cache-ak0.pinimg.com/originals/0d/02/d2/0d02d28791fab2a6f056648dc174033c.jpg'
get_fwhr(bad_img_url_1, url=True)

In [ ]:
bad_img_url_2 = '/Users/carllindersson/Documents/Neurovetenskap/fWHR_Summer/Behaviour_and_statistics/Canada_NHL_images'
get_fwhr(bad_img_url_2, url=True)

# Explanation

The data points used in the functions above use this picture as reference:

In [ ]:
from IPython.display import Image as ImageShow
ImageShow(url = 'https://cdn-images-1.medium.com/max/1600/1*AbEg31EgkbXSQehuNJBlWg.png')